<a href="https://colab.research.google.com/github/akiabe/coursera-dl-tf-in-practice/blob/master/C3W2_Movie_reviw_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [0]:
# Import TensorFlow datasets
import tensorflow_datasets as tfds

# Define imdb set and info set
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [0]:
import numpy as np

# Get imdb training and test data
train_data, test_data = imdb['train'], imdb['test']

# Create sentences and labels list
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Copy the contents of tensors into lists
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# Convert training and testing labels into np.array 
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
# Define parameters
vocab_size = 10000  # Vocabulary size
embedding_dim = 16  # Embedding dimension
max_length = 120    # Maximum of sentense length
trunc_type='post'   # Cut off the back of the review
oov_tok = "<OOV>"   # Define outet of vocabulary token <OOV>


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Instantiate tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
# Tokenizer will then be fit on the training_sentences
tokenizer.fit_on_texts(training_sentences)

# Word index of tokenizer
word_index = tokenizer.word_index
# Convert training_sentences into sequence of numbers
sequences = tokenizer.texts_to_sequences(training_sentences)
# Pad or truncate the sequence of numbers
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Convert testing_sequences into sequence of numbers and padding them
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [33]:
# Take reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Decode review by taking look at numbers in review and reversing that into a word
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [34]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # 2 classes
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Training 10 epochs
num_epochs = 10
model.fit(
    padded,
    training_labels_final,
    epochs=num_epochs,
    validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5102 - accuracy: 0.7275 - val_loss: 0.3438 - val_accuracy: 0.8503
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2353 - accuracy: 0.9099 - val_loss: 0.3674 - val_accuracy: 0.8416
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0825 - accuracy: 0.9800 - val_loss: 0.4679 - val_accuracy: 0.8247
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0210 - accuracy: 0.9974 - val_loss: 0.5242 - val_accuracy: 0.8300
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0065 - accuracy: 0.9995 - val_loss: 0.5841 - val_accuracy: 0.8285
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.6322 - val_accuracy: 0.8294
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.5049e-04 - accuracy: 1.0000 - val_loss: 0.6737 - val_accuracy: 0.8291
Ep

In [36]:
e = model.layers[0]
weights = e.get_weights()[0]  # shape: (vocab_size, embedding_dim)
print(weights.shape) 

(10000, 16)


In [0]:
# Download the value of the 16 dimensions per word as "vecs.tsv" and actual word
# associated with that as "meta.tsv"
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

# Load tsv files to projector.tensorflow.org and can see mapping of words
# into the vector spaces

In [40]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()